# Lab 10.2: Deep Cnn with MNIST Classifier

Here we are going to create the Deep CNN to classify the MNIST Digits.

Edited By Steve Ive

Reference from

## Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn.init as init

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(1)

if device == 'cuda':
    torch.cuda.manual_seed_all(1)

## Set Hyperparameters

In [3]:
learning_rate = 0.001
training_epochs = 20
batch_size = 100

## Load MNIST Data

In [4]:
mnist_train = datasets.MNIST(root = 'MNIST_data/',
                             download = True,
                             train = True,
                             transform = transforms.ToTensor())
mnist_test = datasets.MNIST(root = 'MNIST_data/',
                            download = True,
                            train = False,
                            transform = transforms.ToTensor())

C:\Users\buddhalight\envs\buddhalight\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train, shuffle = True, batch_size = batch_size, drop_last = True)

## Model Define

In [16]:
class Deep_CNN_MNIST(nn.Module):
    def __init__(self):
        super().__init__()
        self.keep_prob = 0.5
        # Layer 1: ImageInput Shape = (?, 1, 28, 28)
        # Conv   : ImageOutput Shape = (?, 32, 28, 28)
        # Pool   : ImageOutput Shape = (?, 32, 14, 14)
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2)
        )
        # Layer 2: ImageInput Shape = (?, 32, 14, 14)
        # Conv   : ImageOutput Shape = (?, 64, 14, 14)
        # Pool   : ImageOutput Shape = (?, 64, 7, 7)
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2)
        )
        # Layer 3: ImageInput Shape = (?, 64, 7, 7)
        # Conv   : ImageOutput Shape = (?, 128, 7, 7)
        # Pool   : ImageOutput Shape = (?, 128, 4, 4)
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 1)
        )
        # Layer 4: Fully Connected 4x4x128 inputs => 625 outputs
        self.fc1 = nn.Linear(4 * 4 * 128, 625)
        init.xavier_uniform_(self.fc1.weight)
        self.layer4 = nn.Sequential(
            self.fc1,
            nn.ReLU(),
            nn.Dropout(p = 1 - self.keep_prob)
        )
        # Layer 5: Final Fully Connected 625 outputs => 10 outputs
        self.layer5 = nn.Linear(625, 10)
        init.xavier_uniform_(self.layer5.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1) # Flatten for Fully Connected
        out = self.layer4(out)
        out = self.layer5(out)
        return out

In [17]:
model = Deep_CNN_MNIST().to(device)

In [18]:
optimizer = optim.Adam(model.parameters(), lr= learning_rate)

## Train

In [19]:
total_batch = len(data_loader)

model.train() # Set Model to train mode
for epoch in range(training_epochs):

    avg_cost = 0

    for X, Y in data_loader:

        # Image is already size of (28 x 28), no reshape
        # Label is not one-hot encoded

        X = X.to(device)
        Y = Y.to(device)

        #prediction
        pred = model(X)

        #cost
        cost = F.cross_entropy(pred, Y).to(device)

        #Reduce cost
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        avg_cost += cost

    avg_cost = avg_cost / total_batch

    print('Epoch: {} / 20, Cost: {:.6f}'.format(epoch + 1, avg_cost))

Epoch: 1 / 20, Cost: 0.188810
Epoch: 2 / 20, Cost: 0.052321
Epoch: 3 / 20, Cost: 0.038685
Epoch: 4 / 20, Cost: 0.029705
Epoch: 5 / 20, Cost: 0.024148
Epoch: 6 / 20, Cost: 0.020392
Epoch: 7 / 20, Cost: 0.016698
Epoch: 8 / 20, Cost: 0.013662
Epoch: 9 / 20, Cost: 0.012959
Epoch: 10 / 20, Cost: 0.013086
Epoch: 11 / 20, Cost: 0.010228
Epoch: 12 / 20, Cost: 0.010833
Epoch: 13 / 20, Cost: 0.008551
Epoch: 14 / 20, Cost: 0.007611
Epoch: 15 / 20, Cost: 0.007190
Epoch: 16 / 20, Cost: 0.006507
Epoch: 17 / 20, Cost: 0.007441
Epoch: 18 / 20, Cost: 0.005674
Epoch: 19 / 20, Cost: 0.004650
Epoch: 20 / 20, Cost: 0.006838


In [26]:
#Test model and Check Accuracy

with torch.no_grad():

    X = mnist_test.data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y = mnist_test.targets.to(device)

    #prediction
    pred = model(X)
    correct_prediction = (torch.argmax(pred, 1) == Y)
    accuracy = correct_prediction.float().mean()
    
    print('Accuracy: ', accuracy.item())

Accuracy:  0.9656999707221985
